### 作業
請嘗試使用 keras 來定義一個直接預測 15 個人臉關鍵點坐標的檢測網路，以及適合這個網路的 loss function


Hint: 參考前面的電腦視覺深度學習基礎

### 範例
接下來的程式碼會示範如何定義一個簡單的 CNN model

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib inline

In [14]:
# 使用 colab 環境的同學請執行以下程式碼
# %tensorflow_version 1.x # 確保 colob 中使用的 tensorflow 是 1.x 版本而不是 tensorflow 2
# import tensorflow as tf
# print(tf.__version__)

# import os
# from google.colab import drive 
# drive.mount('/content/gdrive') # 將 google drive 掛載在 colob，
# %cd 'gdrive/My Drive'
# os.system("mkdir cupoy_cv_part4") # 可以自己改路徑
# %cd cupoy_cv_part4 # 可以自己改路徑

In [15]:
# 讀取資料集以及做前處理的函數
def load_data(dirname):
    # 讀取 csv 文件
    data = pd.read_csv(dirname)
    # 過濾有缺失值的 row
    data = data.dropna()

    # 將圖片像素值讀取為 numpy array 的形態
    data['Image'] = data['Image'].apply(lambda img: np.fromstring(img, sep=' ')).values 

    # 單獨把圖像 array 抽取出來
    imgs = np.vstack(data['Image'].values)/255
    # reshape 為 96 x 96
    imgs = imgs.reshape(data.shape[0], 96, 96)
    # 轉換為 float
    imgs = imgs.astype(np.float32)
    
    # 提取坐標的部分
    points = data[data.columns[:-1]].values

    # 轉換為 float
    points = points.astype(np.float32)

    # normalize 坐標值到 [-0.5, 0.5]
    points = points/96 - 0.5
    
    return imgs, points

In [16]:
# 讀取資料
imgs_train, points_train = load_data(dirname = 'training.csv')
print("圖像資料:", imgs_train.shape, "\n關鍵點資料:", points_train.shape)

圖像資料: (2140, 96, 96) 
關鍵點資料: (2140, 30)


In [17]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU
from keras.optimizers import Adam

In [18]:
# 定義人臉關鍵點檢測網路
model = Sequential()

# 定義神經網路的輸入, hidden layer 以及輸出
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', use_bias=True, name='block1_conv1',input_shape=(96,96,1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', use_bias=True, name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add(Conv2D(128, (1, 1), padding='same', use_bias=True, name='block2_conv1'))
model.add(Conv2D(128, (3, 3), padding='same', use_bias=True, name='block2_conv2'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
model.add(Conv2D(256, (1, 1), padding='same', use_bias=True, name='block3_conv1'))
model.add(Conv2D(256, (3, 3), padding='same', use_bias=True, name='block3_conv2'))
model.add(Conv2D(256, (3, 3), padding='same', use_bias=True, name='block3_conv3'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

    # Block 4
model.add(Conv2D(512, (1, 1), use_bias=True, padding='same', name='block4_conv1'))
model.add(Conv2D(512, (3, 3), use_bias=True, padding='same', name='block4_conv2'))
model.add(Conv2D(512, (3, 3), use_bias=True, padding='same', name='block4_conv3'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))
  
model.add(Flatten(name='flatten'))
model.add(Dense(1024, activation='relu', name='fc1'))
model.add( Dense(1024, activation='relu', name='fc2'))
model.add(Dropout(0.3))
model.add(Dense(30, activation='softmax', name='predictions'))
# 配置 loss funtion 和 optimizer
model.compile(optimizer =Adam(lr=1e-4), loss = 'mean_squared_error')

In [ ]:
model.summary()